REQUIRED DEPENDENCIES
- pandas
- numpy
- matplotlib
- pyarroq
- fastparquet
- dask
- fuzzywuzzy
- ipywidgets
- ipython
- jupyter_contrib_nbextensions

use pip install in terminal

In [ ]:
# If not already installed, do: pip install pandas fastparquet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

url_file = 'pricecatcher/pricecatcher/price_urls.json'
price_df = []


with open(url_file, 'r') as json_file:
    price_urls_data = json.load(json_file)

for entry in price_urls_data:
    parquet_urls = entry['parquet_files']
    for url in parquet_urls:
        df = pd.read_parquet(url)
        price_df.append(df)
    
URL_LOOKUP = 'https://storage.googleapis.com/dosm-public-pricecatcher/lookup_item.parquet'
URL_PREMISE = 'https://storage.googleapis.com/dosm-public-pricecatcher/lookup_premise.parquet'

price = pd.concat(price_df, ignore_index=True)
premise = pd.read_parquet(URL_PREMISE)
lookup = pd.read_parquet(URL_LOOKUP)
if 'date' in price.columns: price['date'] = pd.to_datetime(price['date'])
if 'date' in lookup.columns: lookup['date'] = pd.to_datetime(lookup['date'])
if 'date' in premise.columns: premise['date'] = pd.to_datetime(premise['date'])

In [ ]:
price.shape

LOOKUP TABLE TO BE LEFT JOIN WITH PRICE TABLE

LEFT JOIN LOOKUP TABLE WITH PRICE TABLE ON ITEM CODE

In [ ]:
items=lookup.merge(price,on='item_code',how='left')

In [ ]:
items

LEFT JOIN PREMISE AGAINST PRICE TABLE

In [ ]:
premise

In [ ]:
items_location = premise.merge(price, on='premise_code',how='left',indicator=True)

In [ ]:
items_location

CREATE A MAPPING DICT FROM items_location TABLE WITH  premise_code AS THE KEY AND state AS THE VALUE. UPDATE PREMISE CODE IN items TABLE USING THE MAPPING DICTIONARY

In [ ]:
mapping_location_dict = premise.set_index('premise_code')['state'].to_dict()
items['premise_code'] = items['premise_code'].map(mapping_location_dict)

In [ ]:
lookup_dict = lookup.set_index('item')['item_code'].to_dict()

In [ ]:
memory_usage_bytes = items.memory_usage(deep=True).sum()
print(f"Total memory usage of the DataFrame: {memory_usage_bytes / 1024**3:.2f} GB")

USE VAEX AND DASK BECAUSE THE DATA TABLE IS TOO BIG AND TAKES TOO LONG TO FILTER THE DATA. CONVERT PANDAS DF TO DASK DF SO I CAN USE DASK PARALLEL PROCESSING.

Failed to build vaex-core
ERROR: Could not build wheels for vaex-core, which is required to install pyproject.toml-based projects

At 2 million data point this algorithm can filter data at an average of 40 seconds.
At 44.6 million data points this algorithm can filter data at an average of 720 seconds

For 2 million data points:
Processing Rate = 2,000,000 / 40 = 50,000 data points per second

For 44.6 million data points:
Processing Rate = 44,600,000 / 720 = 61,944 data points per second

In [ ]:
import dask.dataframe as dd
from fuzzywuzzy import fuzz
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
import threading

In [ ]:
def match(input, choices, threshold=80):
        match_score = [(choice, fuzz.partial_ratio(input, choice.lower())) for choice in choices if isinstance(choice, str)]
        matched_item = max(match_score, key=lambda x: x[1], default=None)
        if matched_item[1] >= threshold:
            return matched_item[0]
        else:
            return None
        
def filter_data(dataframe):
    state_input = widgets.Text(description="State: ")
    item_input = widgets.Text(description="Item: ")
    submit_button = widgets.Button(description="Submit")
    cancel_button = widgets.Button(description="Cancel")
    timer = widgets.Label(value="Time elapsed: 0 seconds")
    buttons_box = widgets.HBox([submit_button, cancel_button])

    display(state_input)
    display(item_input)
    display(buttons_box)
    display(timer)

    timer_thread = None
    stop_event = threading.Event()

    def update_timer(start_time, stop_event):
        while True:
            if stop_event.is_set():
                break
            elapsed_time = time.time() - start_time
            timer.value = f"Time elapsed: {elapsed_time: .2f} seconds"
    
    def on_submit(button):
        global filtered_data
        start_time = time.time()
        timer_thread = threading.Thread(target=update_timer, args=(start_time, stop_event))
        timer_thread.start()
        item = item_input.value.strip().lower()
        state = state_input.value.strip().lower()

        if not item:
            print("Item must be entered")
            filtered_data = pd.DataFrame()
        else:
            # Convert pandas DataFrame to Dask DataFrame
            df = dd.from_pandas(dataframe, npartitions=8)

            # Filter the DataFrame using fuzzy matching
            matched_item = match(item, df['item'])
            if matched_item:
                filtered_data = df[df['item'] == matched_item].compute()
                if state:
                    matched_state = match(state, df['premise_code'])
                    if matched_state:
                        item_state = state
                        filtered_data = filtered_data[filtered_data['premise_code'] == matched_state]
            else:
                print("No matches found for the provided item.")
                filtered_data = pd.DataFrame()
            stop_event.set()
            timer_thread.join()
            display(filtered_data)
    

    def on_cancel(button):
        global filtered_data
        print("Action canceled.")
        stop_event.set()
        if timer_thread is not None:
            timer_thread.join()
        filtered_data = pd.DataFrame()

    submit_button.on_click(on_submit)
    cancel_button.on_click(on_cancel)

filter_data(items)

ANALYZE FILTERED DATA

In [ ]:
%matplotlib inline

daily_avg_price = filtered_data.groupby(filtered_data['date'].dt.date)['price'].mean()

item_state = None
item_name = filtered_data.iloc[0]['item']
plt.figure(figsize=(15, 8))
plt.plot(daily_avg_price.index, daily_avg_price.values)

if item_state is not None:
    plt.title(f'Daily Average Price of {item_name} in {item_state}')
else:
    plt.title(f'Daily Average Price of {item_name} in Malaysia')

plt.xlabel('Date')
plt.ylabel('Average Price')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
item_state